## Libraries

In [1]:
# System Utilities
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display

# LLM Utilities
from llama_index.core import Document
from llama_index.core import PromptTemplate
from llama_index.core import Settings
from llama_index.core import ChatPromptTemplate
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core.prompts.prompt_type import PromptType
from llama_index.llms.gemini import Gemini

# Scraping
import re
from llama_index.readers.web import SimpleWebPageReader
from bs4 import BeautifulSoup

# Vector Database
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.gemini import GeminiEmbedding

e:\david\Documents\GitHub\gemini-query-app\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Environment Variables

In [2]:
load_dotenv()

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

## Retriever

### Scraping Website Data

LlamaIndex can be used to load data from different sources. In this case we can use the `SimpleWebPageReader` to retrieve data from a webpage. For more information please visit the [LlamaIndex documentation](https://docs.llamaindex.ai/en/stable/understanding/loading/loading/).

In [3]:
web_documents = SimpleWebPageReader().load_data(
    urls=['https://www.scotiabank.com/ca/en/personal/credit-cards/compare-cards.html']
)

html_content = web_documents[0].text

Now that we have the html content we need to parse the data, retrieving the relevant information we need.

In [4]:
# The soup is the html content structured in HTML format
soup = BeautifulSoup(markup=html_content, features='html.parser')

# The information we're looking for is structured in cards, so we need to find them first
card_content_div = soup.find_all('div', class_='card-content')

credit_cards = []
information = []

# Inside the cards, relevant information is located inside <span> and <p> tags.
for card in card_content_div:
    credit_cards.append(card.find('span', class_='subtitle-1').text)
    information.append(card.find_all('p'))

In [5]:
credit_cards

['Scotiabank Passport® Visa Infinite* Card ',
 'Scotia Momentum® Visa Infinite* Card ',
 'Scotiabank Gold American Express® Card  ',
 'Scotiabank® Scene+™ Visa* Card ',
 'Scotia Momentum® Visa* Card ',
 'Scotia Momentum® No-Fee Visa* Card ',
 'Scotiabank American Express® Card ',
 'Scotiabank Platinum American Express® Card ',
 'Scotia Momentum® Mastercard®* credit card ',
 'Scotiabank® U.S. Dollar Visa* Card ',
 'Scotiabank Value® Visa* Card ',
 'Scotiabank® Scene+™ Visa* Card (for students) ',
 'Scotiabank Value® Visa* Card (for students) ',
 'Scotia Momentum® Visa* Card (for students) ',
 ' Scotia Momentum® No-Fee Visa* Card (for students) ',
 'Scotiabank American Express® Card (for students) ']

This cycle will unify the content of all `<p>` tags inside a String for each card, and then append it to a list

In [6]:
information_text = []

for list in information:
    text = ''
    for tag in list:
        text += tag.text + '\n'
    information_text.append(text)

# This code will remove the '\xa0' elements from the text
information_text = [s.replace('\xa0', ' ') for s in information_text]

In [7]:
information_text

['Earn up to $1,300+ in value in the first 12 months, including up to 40,000 bonus Scene+ points and first year annual fee waived.‡\n \nEarn 3 Scene+ points1 on every $1 you spend at Sobeys, Safeway, IGA, Foodland & Participating Co-ops and more.\n \nAnnual fee: $150\nInterest rates: 20.99% purchases / 22.99% cash advances\n',
 'Earn 10% cash back on all purchases for the first 3 months (up to $2,000 in total purchases).1 Plus, no annual fee in the first year, including on additional cards.1\n \nAnnual fee: $120\nInterest rates: 20.99% purchases / 22.99% cash advances\n',
 'Earn up to $650* in value in the first 12 months, including up to 40,000 bonus Scene+ points.1\n \nEarn 6 Scene+ points2 on every $1 CAD you spend in Canada at Sobeys, Safeway, FreshCo, Foodland and more.\n \nAnnual fee: $120\nInterest rates: 20.99% purchases / 22.99% cash advances\n',
 'Earn up to 7,500 bonus Scene+ points within your first 3 months.2\n \nEarn 2 Scene+ points4 on every $1 you spend at Sobeys, Safew

The following code will combine and add the information of both lists to a string, instantiating a Document object with the `Document` class.

In [8]:
text_content = ''

for i in range(len(credit_cards)):
    text_content += credit_cards[i] + information_text[i]

documents = [Document(text=text_content)]
documents

[Document(id_='ef00233a-38fe-4642-a187-e92b034cf57c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Scotiabank Passport® Visa Infinite* Card Earn up to $1,300+ in value in the first 12 months, including up to 40,000 bonus Scene+ points and first year annual fee waived.‡\n \nEarn 3 Scene+ points1 on every $1 you spend at Sobeys, Safeway, IGA, Foodland & Participating Co-ops and more.\n \nAnnual fee: $150\nInterest rates: 20.99% purchases / 22.99% cash advances\nScotia Momentum® Visa Infinite* Card Earn 10% cash back on all purchases for the first 3 months (up to $2,000 in total purchases).1 Plus, no annual fee in the first year, including on additional cards.1\n \nAnnual fee: $120\nInterest rates: 20.99% purchases / 22.99% cash advances\nScotiabank Gold American Express® Card  Earn up to $650* in value in the first 12 months, including up to 40,000 bonus Scene+ points.1\n \nEarn 6 Scene+ points2 on every $1 CAD you s

### Initializing Models

Embeddings Model (**Embedding-001**)

In [9]:
gemini_embedding_model = GeminiEmbedding(api_key=GEMINI_API_KEY, model_name='models/embedding-001')

LLM Model (**Gemini Pro**)

In [10]:
llm = Gemini(api_key=GEMINI_API_KEY, model_name='models/gemini-pro')

### Store Data with Chroma

In [11]:
client = chromadb.PersistentClient(path='./chroma_db')
chroma_collection = client.get_or_create_collection('credit_cards')

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

Settings.llm = llm
Settings.embed_model = gemini_embedding_model

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

### Retriever Creation

In [12]:
load_client = chromadb.PersistentClient(path='./chroma_db')

chroma_collection = load_client.get_collection('credit_cards')

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(vector_store)

In [45]:
test_query_engine = index.as_query_engine()
response = test_query_engine.query('What are the benefits of the Passport Visa Infinite card?')
print(response)

Earn up to $1,300+ in value in the first 12 months, including up to 40,000 bonus Scene+ points and first year annual fee waived.‡
Earn 3 Scene+ points1 on every $1 you spend at Sobeys, Safeway, IGA, Foodland & Participating Co-ops and more.


## Generator

### Prompt Templates

In [37]:
chat_engine = index.as_chat_engine(
    chat_mode='context',
    system_prompt=(
        "You are a chatbot, able to have normal interactions, as well as"
        " provide information about credit cards using only"
        " the context information below.\n"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "Given the context information and not prior knowledge, "
        "answer the question.\n"
        "Question: {query_str}\n"
        "Answer: "
    )
)

In [42]:
response = chat_engine.chat('hello!')
print(response)

Hello! How can I help you today? I can provide information about credit cards, or we can just chat.


In [43]:
response = chat_engine.chat('Which credit card has the best benefits with an annual fee under 140 dollars?')
print(response)

The Scotia Momentum® Visa Infinite* Card has the best benefits with an annual fee under $140. It offers 10% cash back on all purchases for the first 3 months (up to $2,000 in total purchases), plus no annual fee in the first year, including on additional cards. After the first year, the annual fee is $120. Other benefits include:

* Earn 2% cash back on all purchases after the first 3 months
* No foreign transaction fees
* Travel insurance
* Purchase protection
* Extended warranty protection


In [35]:
response = chat_engine.chat('Which credit card has the lowest interest rates?')
print(response)

Scotiabank Platinum American Express® Card has the lowest interest rates at 9.99% for both purchases and cash advances.


In [24]:
response = chat_engine.chat('Which credit cards have 0 annual fee?')
print(response)

The following credit cards have 0 annual fee:

* Scotiabank® Scene+™ Visa* Card (for students)
* Scotia Momentum® No-Fee Visa* Card (for students)
* Scotiabank American Express® Card (for students)
* Scotia Momentum® Mastercard®* credit card


In [25]:
response = chat_engine.chat('Okey, and what credit card do you recommend me if I would like to get a lot of Scene+ points?')
print(response)

If you're looking to get a lot of Scene+ points, I recommend the Scotiabank Gold American Express® Card. With this card, you can earn up to $650* in value in the first 12 months, including up to 40,000 bonus Scene+ points. You'll also earn 6 Scene+ points on every $1 CAD you spend in Canada at Sobeys, Safeway, FreshCo, Foodland and more.


In [26]:
response = chat_engine.chat('What credit card would you recommend to me if this is the first time I am acquiring one?')
print(response)

If this is your first time getting a credit card, I recommend the Scotia Momentum® No-Fee Visa* Card (for students). This card has no annual fee and offers a 0% introductory interest rate on balance transfers for the first 6 months (22.99% after that). You'll also earn 5% cash back on all purchases for the first 3 months (up to $2,000 in total purchases). This card is a great way to build your credit and start earning rewards.


In [27]:
response = chat_engine.chat('Can you show me other suitable options?')
print(response)

Sure. Here are a few other credit cards that may be suitable for you if you're a student:

* **Scotiabank Value® Visa* Card (for students)**: This card has a low annual fee of $29 and offers a 0% introductory interest rate on balance transfers for the first 10 months (12.99% and then after that 13.99% starting August 1, 2024). You'll also earn 2 Scene+ points on every $1 you spend at Sobeys, Safeway, Foodland & Participating Co-ops, FreshCo and more.
* **Scotia Momentum® Visa* Card (for students)**: This card has an annual fee of $39 and offers a 2.99% introductory interest rate on balance transfers for the first 6 months (22.99% after that). You'll also earn 10% cash back on all purchases for the first 3 months (up to $2,000 in total purchases).
* **Scotiabank American Express® Card (for students)**: This card has no annual fee and offers up to 7,500 bonus Scene+ points within your first 3 months (that's up to $75 towards travel). You'll also earn 3 Scene+ points on every $1 you spend

In [28]:
response = chat_engine.chat('What of those you suggest me to acquire if I need to make a balance transfer?')
print(response)

If you need to make a balance transfer, I recommend the **Scotiabank Value® Visa* Card (for students)**. This card has a 0% introductory interest rate on balance transfers for the first 10 months (12.99% and then after that 13.99% starting August 1, 2024). This means that you can transfer your balance from another credit card to this card and pay it off interest-free for 10 months. This can save you a lot of money on interest charges.


In [29]:
response = chat_engine.chat('I travel frequently to United States for shopping. Which credit card do you recommend to me?')
print(response)

If you travel frequently to the United States for shopping, I recommend the **Scotiabank® U.S. Dollar Visa* Card**. This card has no currency conversion fees, so you won't have to pay extra when you make purchases in U.S. dollars. The card also has a competitive interest rate, so you can save money on interest charges if you carry a balance.


In [30]:
response = chat_engine.chat('What credit card could be useful for getting a lot of cashback?')
print(response)

If you're looking to get a lot of cashback, I recommend the **Scotia Momentum® Visa* Card**. This card offers 10% cash back on all purchases for the first 3 months (up to $2,000 in total purchases). After that, you'll earn 1% cash back on all eligible gas station, grocery store, drug store purchases and recurring payments.


In [31]:
response = chat_engine.chat('Which credit card is best for me if I carry with high balances?')
print(response)

If you carry a high balance on your credit card, I recommend the **Scotiabank Value® Visa* Card**. This card has a low interest rate of 12.99% on purchases and 12.99% on cash advances. This means that you'll pay less interest on your balance than you would with other cards.


In [32]:
response = chat_engine.chat('There is another card with equal or lower interest rates?')
print(response)

Yes, there is one other card with an equal interest rate:

* **Scotia Momentum® Mastercard®* credit card: This card also has an interest rate of 12.99% on purchases and 12.99% on cash advances.


In [33]:
response = chat_engine.chat('Please provide me the information about the Platinum card')
print(response)

**Scotiabank Platinum American Express® Card**

* Earn up to $2,100* in value in the first 14 months, including up to 60,0004 bonus Scene+ points.
* Earn 2X the Scene+ points1 for every $1 you spend.
* Annual fee: $399
* Interest rates: 9.99% purchases / 9.99% cash advances


In [41]:
chat_engine.reset()